In [1]:
import os
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [2]:
root = "/raid/ly/Dataset/tianchi2021/user_data/tmp_data/tracking/MOT-PANDA/"
out_root = "/raid/ly/Dataset/tianchi2021/user_data/tmp_data/classification/"
vid_nms = [x for x in os.listdir(root)]

In [3]:
ann_paths = [os.path.join(root, vid_nm)+'/gt/gt.txt' for vid_nm in vid_nms]
img_roots = [os.path.join(root, vid_nm)+'/img1' for vid_nm in vid_nms]

In [4]:
for i in tqdm(range(len(ann_paths))):
    ann_path = ann_paths[i]
    img_root = img_roots[i]
    seq_id = ann_path.split('/')[-3].split('_')[0]
    ann_df = pd.read_csv(ann_path, names=['frame', 'id', 'bb_left', 'bb_top', 'bb_width', 'bb_height', 'conf', 'x', 'y', 'z', 'occ'])
    ann_df.fillna("disappear", inplace=True)
    for fid, group in tqdm(ann_df.groupby('frame'), total = len(ann_df.groupby('frame'))):
        img_nm = '_'.join(['SEQ',seq_id, str(fid).zfill(3)]) + '.jpg'
        img_path = os.path.join(img_root, img_nm)
        img = cv2.imread(img_path)
        img_h, img_w, _ = img.shape
        for ann_id, ann in group.iterrows():
            x, y, w, h = ann.bb_left, ann.bb_top, ann.bb_width, ann.bb_height
            fid, pid = ann.frame, ann.id
            x, y, w, h = int(x), int(y), int(w), int(h)
            if x < 0:
                x = 0
            if y < 0:
                y = 0
            if x + w > img_w:
                w = img_w - x -1
            if y + h > img_h:
                y = img_h - h -1     
            if h * w > 5:
                sub_img = img[y:y+h, x:x+w, :]
                sub_img_nm = "_".join([str(x) for x in [seq_id, fid, pid, x, y, w, h]]) + '.jpg'
                sub_img_out_path = os.path.join(out_root, ann.occ, sub_img_nm)
                cv2.imwrite(sub_img_out_path, sub_img)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/234 [00:00<?, ?it/s]

  0%|          | 0/234 [00:00<?, ?it/s]

  0%|          | 0/234 [00:00<?, ?it/s]

  0%|          | 0/234 [00:00<?, ?it/s]

...

  0%|          | 0/180 [00:00<?, ?it/s]

  0%|          | 0/148 [00:00<?, ?it/s]

  0%|          | 0/234 [00:00<?, ?it/s]

  0%|          | 0/234 [00:00<?, ?it/s]

  0%|          | 0/133 [00:00<?, ?it/s]

In [9]:
occ_img_root = "../../user_data/tmp_data/classification/"
occ_cls = ['disappear', 'hide', 'normal', 'serious hide']
occ_img_paths = []
for cls in occ_cls:
    img_root =os.path.join(occ_img_root, cls)
    img_nms = os.listdir(img_root)
    occ_img_paths += [os.path.join(img_root, img_nm) for img_nm in img_nms]

In [26]:
train_keep_root = "../../user_data/tmp_data/classification/train/keep/"
train_drop_root = "../../user_data/tmp_data/classification/train/drop/"
val_keep_root = "../../user_data/tmp_data/classification/val/keep/"
val_drop_root = "../../user_data/tmp_data/classification/val/drop/"

trainsub_keep_root = "../../user_data/tmp_data/classification/occ_data_sub/train/keep/"
trainsub_drop_root = "../../user_data/tmp_data/classification/occ_data_sub/train/drop/"
valsub_keep_root = "../../user_data/tmp_data/classification/occ_data_sub/val/keep/"
valsub_drop_root = "../../user_data/tmp_data/classification/occ_data_sub/val/drop/"

In [19]:
len(occ_img_paths)

695018

In [13]:
import shutil

In [18]:
for occ_img_path in tqdm(occ_img_paths, total = len(occ_img_paths)):
    img_nm = occ_img_path.split('/')[-1].split('_')[0]
    if (('disappear' in occ_img_path) | ('serious hide' in occ_img_path)) & (img_nm != '10'):
        dst = train_drop_root
    elif (('normal' in occ_img_path) | ('hide' in occ_img_path)) & (img_nm != '10'):
        dst = train_keep_root
    elif (('disappear' in occ_img_path) | ('serious hide' in occ_img_path)) & (img_nm == '10'):
        dst = val_drop_root
    elif (('normal' in occ_img_path) | ('hide' in occ_img_path)) & (img_nm == '10'):
        dst = val_keep_root
    
    shutil.copy2(occ_img_path, dst)

  0%|          | 0/695018 [00:00<?, ?it/s]

In [29]:
for occ_img_path in tqdm(occ_img_paths, total = len(occ_img_paths)):
    img_nm = occ_img_path.split('/')[-1].split('_')[0]
    fid = occ_img_path.split('/')[-1].split('_')[1]
    if (('disappear' in occ_img_path) | ('serious hide' in occ_img_path)) & (img_nm != '10') & (int(fid) % 5 == 0):
        dst = trainsub_drop_root
        shutil.copy2(occ_img_path, dst)
    elif (('normal' in occ_img_path) | ('hide' in occ_img_path)) & (img_nm != '10') & (int(fid) % 5 == 0):
        dst = trainsub_keep_root
        shutil.copy2(occ_img_path, dst)
    elif (('disappear' in occ_img_path) | ('serious hide' in occ_img_path)) & (img_nm == '10') & (int(fid) % 5 == 0):
        dst = valsub_drop_root
        shutil.copy2(occ_img_path, dst)
    elif (('normal' in occ_img_path) | ('hide' in occ_img_path)) & (img_nm == '10') & (int(fid) % 5 == 0):
        dst = valsub_keep_root
        shutil.copy2(occ_img_path, dst)

  0%|          | 0/695018 [00:00<?, ?it/s]